In [1]:
import shutil
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup as bs
import time
import random
import pandas as pd
from openpyxl import Workbook
import openpyxl
import os

delay = random.uniform(1, 2.1)
options = Options()

data = []
fail_list = []

previous_cik = None
previous_sheetname_lists = None

def get_symbol(cik, symbol_df):
    match = symbol_df[symbol_df["CIK"] == cik]
    if not match.empty:
        return match["Symbol"].values[0]
    else:
        return ""

def excel_sheet_modify(open_path, symbol_df, doc_type):
    sheet_data = []
    df = pd.read_excel(open_path)
    
    if doc_type == "10_K":
        all_list_df = df[["Symbol", "CIK", "Filling Date", "Link"]].dropna()
    else:  # 10-Q
        all_list_df = df[["CIK", "Filling Date", "Link"]].dropna()
        all_list_df["Symbol"] = all_list_df["CIK"].apply(lambda cik: get_symbol(cik, symbol_df))
    
    finish_count = 0
    global previous_cik, previous_sheetname_lists
    for index, row in all_list_df[2742:].iterrows():
        cik = row["CIK"]
        filing_date = row["Filling Date"]
        link = row["Link"]
        symbol_row = row["Symbol"]
        filtered_sheet_names = []

        if symbol_row:
            symbol_select = symbol_row
            try:
                symbol = symbol_select.split("/")[0]
            except:
                symbol = symbol_select 
        else:
            symbol = "NA"
        
        if int(filing_date.split("-")[0]) <= 2013:
            print(f"Skipping file due to date filter: {symbol}_{cik}_{filing_date}_{doc_type}.xlsx")
            continue
            
        if doc_type == "10_K":
            symbol_folder_path = f"{symbol}_{int(cik)}\\"
            open_excel_file_name = f"{symbol}_{int(cik)}_{filing_date}_{doc_type}.xlsx"
            new_folder_name = f"{symbol}_{cik}_filtered"
        else:  # 10-Q
            symbol_folder_path = f"{symbol}_{int(cik)}\\"
            open_excel_file_name = f"{symbol}_{int(cik)}_{filing_date}_{doc_type}.xlsx"
            new_folder_name = f"{symbol}_{cik}_filtered"
        
        try:
            full_path = os.path.join(excel_open_path, symbol_folder_path, open_excel_file_name)
            if not os.path.exists(full_path):
                print(f"File not found: {full_path}")
                fail_list.append(open_excel_file_name)
                continue
            
            modify_excel = openpyxl.load_workbook(full_path)
            
            if not modify_excel.sheetnames:
                print(f"{open_excel_file_name} 파일이 없습니다.")
                fail_list.append(open_excel_file_name)
                continue
            
            if cik == previous_cik:
                sheetname_lists = previous_sheetname_lists
            else:
                sheetname_lists = sheetname(link)f
                previous_cik = cik
                previous_sheetname_lists = sheetname_lists
         
            for sheet in modify_excel.sheetnames:
                ws = modify_excel[sheet]
                a1_value = ws['A1'].value
                sheet_data.append({'origin Sheet Name': sheet, 'A1 Value': a1_value})
                
            df_sheet_data = pd.DataFrame(sheet_data)   
            filtered_df = df_sheet_data[df_sheet_data['A1 Value'].apply(startswith_any, compare_list=sheetname_lists)]
            filtered_sheet_names = filtered_df['origin Sheet Name'].tolist()
            filtered_workbook = modify_excel
        
        except Exception as e:
            print(e)
            fail_list.append(open_excel_file_name)
            continue
            
        target_folder_path = os.path.join(target_folder_base_path, new_folder_name)
        if not os.path.exists(target_folder_path):
            os.makedirs(target_folder_path)
            
        new_file_name = f"{symbol}_{cik}_{filing_date}_{doc_type}_filtered.xlsx"
        new_file_path = os.path.join(target_folder_path, new_file_name)

        if 'filtered_workbook' in locals():
            first_sheet_name = filtered_workbook.sheetnames[0]
            
            # 유효 시트 제외 삭제
            filter_count = 0
            for sheet in filtered_workbook.sheetnames:
                if sheet not in filtered_sheet_names and sheet != first_sheet_name:
                    std = filtered_workbook[sheet]
                    filtered_workbook.remove(std)
                else:
                    print(f"filtering successful!!! <{sheet}>")
                    filter_count += 1
            
            if filter_count == len(sheetname_lists) + 1:
                if filter_count == 1:
                    print(f"Retrying with sheetname function: {open_excel_file_name}")
                    sheetname_lists = sheetname(link)
                    previous_cik = cik
                    previous_sheetname_lists = sheetname_lists
                else:
                    print(f"successful : {new_file_path}, {finish_count}/{len(all_list_df)} for {doc_type}")
                     # 필터링 파일 다른이름 저장
                    filtered_workbook.save(new_file_path)
                    finish_count += 1

            else:
                print(f"Retrying with sheetname function: {open_excel_file_name}")
                sheetname_lists = sheetname(link)
                previous_cik = cik
                previous_sheetname_lists = sheetname_lists
                
                sheet_data = []
                for sheet in modify_excel.sheetnames:
                    ws = modify_excel[sheet]
                    a1_value = ws['A1'].value
                    sheet_data.append({'origin Sheet Name': sheet, 'A1 Value': a1_value})
                    
                df_sheet_data = pd.DataFrame(sheet_data)   
                filtered_df = df_sheet_data[df_sheet_data['A1 Value'].apply(startswith_any, compare_list=sheetname_lists)]
                filtered_sheet_names = filtered_df['origin Sheet Name'].tolist()
                
                filter_count = 0
                for sheet in filtered_workbook.sheetnames:
                    if sheet not in filtered_sheet_names and sheet != first_sheet_name:
                        std = filtered_workbook[sheet]
                        filtered_workbook.remove(std)
                    else:
                        print(f"filtering successful!!! <{sheet}>")
                        filter_count += 1
                
                if filter_count == len(sheetname_lists) + 1:
                    print(f"successful : {new_file_path}, {finish_count}/{len(all_list_df)} for {doc_type}")
                    filtered_workbook.save(new_file_path)
                    finish_count += 1
                else:
                    print(f"Copying original file due to fail filtering: {open_excel_file_name}")
                    shutil.copy(full_path, new_file_path)
                    continue
                
            print(f"total filtering sheet : {filter_count}/{len(sheetname_lists)+1}")
           
        else:
            # 원본 파일 복사
            print(f"Copying original file due to error: {open_excel_file_name}")
            shutil.copy(full_path, new_file_path)

def sheetname(link_url):
    if link_url:
        financial_statements_sheetnames = []
        user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
        options.add_argument(f"user-agent={user_agent}")
        # options.add_argument("headless")
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
        time.sleep(delay)
        head_url = "https://www.sec.gov"
        url = head_url + link_url
        driver.get(url)
        time.sleep(delay)
        html = driver.page_source
        detail_page = bs(html, "html.parser")
        driver.close()
        financial_statements_tags = detail_page.find_all("li", class_="accordion")
    
        for tag in financial_statements_tags:
            if tag.find("a") and tag.find("a").get_text(strip=True) == "Financial Statements":
                ul_tag = tag.find("ul")
                if ul_tag:
                    a_tags = ul_tag.find_all("a", class_="xbrlviewer")
                    financial_statements_sheetnames.extend([a.get_text(strip=True) for a in a_tags])
    else:
        print(f"Non link...{finish_count-1}")    
    return financial_statements_sheetnames

def startswith_any(value, compare_list):
    if value is None:
        return False
    value = normalize_value(value)
    return any(value.startswith(normalize_value(prefix)) for prefix in compare_list)

def normalize_value(value):
    if isinstance(value, str):
        return value.strip().lower()
    return value

open_path_10k = "C:\\SEC_Data\\sec_edgar_final_filtered\\sec_edgar_bank_section_10_k.xlsx"
open_path_10q = "C:\\SEC_Data\\sec_edgar_final_filtered\\sec_edgar_bank_section_10_q.xlsx"
excel_open_path = "C:\\SEC_Data\\sec_edgar_final_filtered\\origin\\"
target_folder_base_path = "C:\\SEC_Data\\sec_edgar_final_filtered\\filtered\\"

# open_path_10k = "C:\\SEC_Data\\test_folder\\test_url_10k.xlsx"
# open_path_10q = "C:\\SEC_Data\\test_folder\\test_url_10q.xlsx"
# excel_open_path = "C:\\SEC_Data\\test_folder\\add_test\\"
# target_folder_base_path = "C:\\SEC_Data\\test_folder\\filtered\\"


# 10-K 문서 처리
# doc_type = "10_K"
symbol_df = pd.read_excel(open_path_10k)[["CIK", "Symbol"]].drop_duplicates()
# excel_sheet_modify(open_path_10k, symbol_df, doc_type)
# print(f"fail file name: {fail_list}")

# 10-Q 문서 처리
doc_type = "10_Q"
excel_sheet_modify(open_path_10q, symbol_df, doc_type)
print(f"fail file name: {fail_list}")


filtering successful!!! <Cover Page>
filtering successful!!! <CONSOLIDATED STATEMENTS OF INCO>
filtering successful!!! <CONSOLIDATED STATEMENTS OF COMP>
filtering successful!!! <CONSOLIDATED BALANCE SHEETS (UN>
filtering successful!!! <CONSOLIDATED BALANCE SHEETS (_2>
filtering successful!!! <CONSOLIDATED STATEMENTS OF CHAN>
filtering successful!!! <CONSOLIDATED STATEMENTS OF CH_2>
filtering successful!!! <CONSOLIDATED STATEMENTS OF CASH>
successful : C:\SEC_Data\sec_edgar_final_filtered\filtered\COF_927628_filtered\COF_927628_2024-05-02_10_Q_filtered.xlsx, 0/4028 for 10_Q
total filtering sheet : 8/8
filtering successful!!! <Cover Page>
filtering successful!!! <CONSOLIDATED STATEMENTS OF INCO>
filtering successful!!! <CONSOLIDATED STATEMENTS OF COMP>
filtering successful!!! <CONSOLIDATED STATEMENTS OF CHAN>
filtering successful!!! <CONSOLIDATED STATEMENTS OF CH_2>
filtering successful!!! <CONSOLIDATED STATEMENTS OF CASH>
Retrying with sheetname function: COF_927628_2023-11-02_10_Q.xlsx